In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import numpy as np
from keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU


In [ ]:
file_path = '/content/Drying3.xlsx'
data = pd.read_excel(file_path, sheet_name='Sheet1')

X = data[['Drying Time (min )', 'Velocity m/s', 'Temperatures C', 'Thickness mm', 'Sample mass in (g)']]
y = data[['M.C d.b.%']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
model_8_layers = Sequential([
    Dense(64, input_shape=(X_train_scaled.shape[1],)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    Dropout(0.2),

    Dense(128),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    Dropout(0.2),

    Dense(y_train.shape[1], activation='linear')
])


In [ ]:
reduce_lr_8 = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=8, min_lr=1e-6)
early_stop_8 = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
model_8_layers.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])


In [ ]:

history = model_8_layers.fit(X_train_scaled, y_train,
                    epochs=1000,
                    batch_size=32,
                    validation_split=0.2,
                    verbose=1,
                   )

y_pred = model_8_layers.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Error (MAE):", mae)
print("R² Score:", r2)

model_8_layers.save('final_dnn_model.keras')



Epoch 1/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - loss: 2.2396 - mae: 1.2039 - val_loss: 0.3510 - val_mae: 0.5402
Epoch 2/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.2950 - mae: 0.9336 - val_loss: 0.1995 - val_mae: 0.3858
Epoch 3/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.6274 - mae: 0.6189 - val_loss: 0.1420 - val_mae: 0.2986
Epoch 4/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.4051 - mae: 0.5058 - val_loss: 0.1218 - val_mae: 0.2740
Epoch 5/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.5479 - mae: 0.5619 - val_loss: 0.1078 - val_mae: 0.2611
Epoch 6/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.3682 - mae: 0.4727 - val_loss: 0.0960 - val_mae: 0.2498
Epoch 7/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.4933 - mae: 0.5205 - val_loss: 0.1011 - val_mae: 0.2558
Epoch 8/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.3338 - mae: 0.4352 - val_loss: 0.1227 - val_mae: 0.2828
Epoch 9/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.3875 -

In [7]:
import numpy as np

def predict_new(data):
    data_scaled = scaler.transform([data])
    prediction = model_8_layers.predict(data_scaled)
    return [f"{val:.4f}" for val in prediction[0]]  # تحويل القيم إلى نص وتنسيقها لأربعة أرقام عشرية

# بيانات جديدة للتنبؤ
data_example = [210, 2, 80, 7, 488.16]
prediction = predict_new(data_example)

# عرض النتائج
print("Predictions for new data:", prediction)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Predictions for new data: ['0.0072']


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
